# Intro to `pandas` for machine learning

We'll explore the Pandas package for simple data handling tasks using geoscience data examples. 

Pandas introduces the concept of a `DataFrame` in Python. If you're familiar with R, it's pretty much the same idea! Useful cheat sheet [here](https://www.datacamp.com/community/blog/pandas-cheat-sheet-python#gs.59HV6BY)

The main purpose of Pandas is to allow easy manipulation of data in tabular form. Perhaps the most important idea that makes Pandas great for data science, is that it will always preserve **alignment** between data and labels.

In [1]:
import pandas as pd

The most common data structure in Pandas is the `DataFrame`, which is a 2D structure that can hold various types of Python objects indexed by an `index` array (or multiple `index` arrays). Columns are usually labelled as well using strings. A column on its own is a different data type, called a `Series`.

An easy way to think about a `DataFrame` is if you imagine it as an Excel spreadsheet.

Let's define one using a small dataset:

In [2]:
data =  [[2.13, 'sandstone'],
         [3.45, 'limestone'],
         [2.45, 'shale']]
data

[[2.13, 'sandstone'], [3.45, 'limestone'], [2.45, 'shale']]

Make a `DataFrame` from `data`

In [3]:
df = pd.DataFrame(data, columns=['velocity', 'lithology'])
df

,velocity,lithology
0,2.13,sandstone
1,3.45,limestone
2,2.45,shale


Accessing the data is a bit more complex than in the numpy array cases but for good reasons

In [4]:
df['lithology']

0    sandstone
1    limestone
2        shale
Name: lithology, dtype: object

In [5]:
df[0]  # Can't get at rows like this.

KeyError: 0

In [6]:
df.loc[0:1]  # Inclusive slice, unlike anything else in Python or NumPy

,velocity,lithology
0,2.13,sandstone
1,3.45,limestone


In [7]:
df['id'] = [101, 102, 103]
df = df.set_index('id')
df.head()

,velocity,lithology
id,,
101,2.13,sandstone
102,3.45,limestone
103,2.45,shale


In [8]:
df.loc[0]

KeyError: 0

In [9]:
df.iloc[0:1]  # Works like NumPy, rarely needed

,velocity,lithology
id,,
101,2.13,sandstone


In [10]:
df.loc[df['velocity'] < 3]  # Loc provides *selectors* like NumPy arrays.

,velocity,lithology
id,,
101,2.13,sandstone
103,2.45,shale


In [11]:
df.loc[df['velocity'] < 3, 'lithology']  # Both rows and columns.


id
101    sandstone
103        shale
Name: lithology, dtype: object

## Adding data

Add more data (row wise)

In [12]:
df.loc[99] = [3.5, 'dolomite']

In [13]:
df.loc[3] = [2.6, 'shale']
df

,velocity,lithology
id,,
101,2.13,sandstone
102,3.45,limestone
103,2.45,shale
99,3.50,dolomite
3,2.60,shale


Add a new column with a "complete" list, array or series. Alternatively, you can broadcast a value or calculation.

In [14]:
df['new_column'] = ['x', 'y', 'z', 'a', 'b']
df

,velocity,lithology,new_column
id,,,
101,2.13,sandstone,x
102,3.45,limestone,y
103,2.45,shale,z
99,3.50,dolomite,a
3,2.60,shale,b


In [15]:
df['source'] = 'Agile'
df['velocity_ms'] = df['velocity'] * 1000
df

,velocity,lithology,new_column,source,velocity_ms
id,,,,,
101,2.13,sandstone,x,Agile,2130.0
102,3.45,limestone,y,Agile,3450.0
103,2.45,shale,z,Agile,2450.0
99,3.50,dolomite,a,Agile,3500.0
3,2.60,shale,b,Agile,2600.0


In [18]:
df.loc[df.lithology=='shale', 'velocity'] = df.velocity * 1000
df

,velocity,lithology,new_column,source,velocity_ms
id,,,,,
101,2.13,sandstone,x,Agile,2130.0
102,3.45,limestone,y,Agile,3450.0
103,2450.00,shale,z,Agile,2450.0
99,3.50,dolomite,a,Agile,3500.0
3,2600.00,shale,b,Agile,2600.0


## Column types

It's especially worth checking that your numbers are actually numbers (eg floats or ints), because they look the same as strings in the various renderings of DataFrames and Series.

In [16]:
s = pd.Series([23, '23'])
s

0    23
1    23
dtype: object

In [17]:
s * 2

0      46
1    2323
dtype: object

Our DataFrame is not too bad already...

In [18]:
df.dtypes

velocity       float64
lithology       object
new_column      object
source          object
velocity_ms    float64
dtype: object

It makes sense to use the [Pandas `categorical` type](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) for categorical variables. Let's do that:

In [19]:
cols = ['lithology', 'new_column']
df[cols] = df[cols].apply(pd.Categorical)

df.dtypes

velocity        float64
lithology      category
new_column     category
source           object
velocity_ms     float64
dtype: object

Although strings are added as `object` by default, there is a new-ish (in v1.0) data type for text data, `pd.StringDtype()`.

In [20]:
density = [
    'Medium',
    'Low',
    'Med',
    3214,
    'High',
]

First we'll try accepting the default `object` data type:

In [21]:
s = pd.Series(density)  # Or pd.StringDtype()

In [22]:
s.str.isdigit()

0    False
1    False
2    False
3      NaN
4    False
dtype: object

In [23]:
s.str.replace('Med$', 'Medium', regex=True)

0    Medium
1       Low
2    Medium
3       NaN
4      High
dtype: object

Note that the indices of this list do not match the DataFrame index, so we have to explicitly pass the index to the Series constructor. (Try leaving `index=df.index` out.)

In [24]:
s = pd.Series(density, dtype='string')  # Or pd.StringDtype()

In [25]:
s.dtype

string[python]

Using this does not, however, give us direct access to string methods — e.g. you still need `str` here — but at least things work more consistently:

In [26]:
s.str.replace('Med$', 'Medium', regex=True)

0    Medium
1       Low
2    Medium
3      3214
4      High
dtype: string

Some things break when there's a column with mixed content:

In [27]:
s.str.isdigit()

0    False
1    False
2    False
3     True
4    False
dtype: boolean

## Reading and writing files

Pandas also reads files from disk in tabular form ([here](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html)'s a list of all the formats that it can read and write). A very common one is CSV, so let's load one!

The data is the same as used in this study: http://www.kgs.ku.edu/PRS/publication/2003/ofr2003-30/index.html

From that poster:

> The Panoma Field (2.9 TCF gas) produces from Permian Council Grove Group marine carbonates and nonmarine silicilastics in the Hugoton embayment of the Anadarko Basin. It and the Hugoton Field, which has produced from the Chase Group since 1928, the top of which is 300 feet shallower have combined to produce 27 TCF gas, making it the largest gas producing area in North America. Both fields are stratigraphic traps with their updip west and northwest limits nearly coincident. Maximum recoveries in the Panoma are attained west of center of the field. Deeper production includes oil and gas from Pennsylvanian Lansing-Kansas City, Marmaton, and Morrow and the Mississippian.

In [83]:
df = pd.read_csv('https://geocomp.s3.amazonaws.com/data/Panoma_Field_Permian_RAW.csv')
df.head()

,Index,Well Name,Depth,Formation,RelPos,Marine,GR,ILD,DeltaPHI,PHIND,PE,Facies,Completion Date,Source
0,63,SHRIMPLIN,851.3064,A1 SH,1.000,1.0,77.45,4.6132,9.9,11.915,4.6,Nonmarine fine siltstone,2010-03-26,KGS
1,64,SHRIMPLIN,851.4588,A1 SH,0.979,1.0,78.26,4.5814,14.2,12.565,4.1,Nonmarine fine siltstone,2010-03-26,KGS
2,65,SHRIMPLIN,851.6112,A1 SH,0.957,1.0,79.05,4.5499,14.8,13.050,3.6,Nonmarine fine siltstone,2010-03-26,KGS
3,66,SHRIMPLIN,851.7636,A1 SH,0.936,1.0,86.10,4.5186,13.9,13.115,3.5,Nonmarine fine siltstone,2010-03-26,KGS
4,67,SHRIMPLIN,851.9160,A1 SH,0.915,1.0,74.58,4.4361,13.5,13.300,3.4,Nonmarine fine siltstone,2010-03-26,KGS


Writing data out is similarly simple:

In [ ]:
df.to_csv('../data/Panoma_Field_Permian-RAW.csv')

If we are comfortable with SQL, or have an existing database, we may wish to write our dataframe as a table there. We will use the Python implementation of [sqlite](https://www.sqlite.com/index.html), [sqlite3](https://docs.python.org/3/library/sqlite3.html). If you have an existing database you may prefer to look at [SQLalchemy](https://docs.sqlalchemy.org/) to create the connection instead:

In [ ]:
import sqlite3

In [ ]:
connection = sqlite3.connect('../data/panoma.db')
df.to_sql('panoma_raw', con=connection, if_exists='replace', index=False)

You could use `read_sql` to get data from a SQL database instead of reading a file.

<hr />

<p style="color:gray">©2022 Agile Geoscience. Licensed CC-BY.</p>